In [6]:
!pip install kfp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 5.7 MB/s eta 0:00:003.1 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.0 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.1 MB/s eta 0:00:00
Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 19.4 MB/s eta 0:00:00
  Created wheel for kfp: filename=kfp-2.7.0-py3-none-any.whl size=610419 sha256=754eb85ceb87f57956cede28629483bb4c80e07c5afff3771e5b54d8d42f87ea
  Stored in directory: /Users/goutham/Library/Caches/pip/wheels/6b/21/04/3a90a57cfed7fb3b132d75ef1

In [7]:
from kfp import dsl

In [8]:
from kfp import compiler

In [13]:
@dsl.component
def say_hello(name: str) -> str:
    hello_text = f'Hello, {name}'
    return hello_text

In [14]:
@dsl.component
def how_are_you(hello_text: str) -> str:
    how_are_you = f"{hello_text}. How are you?"
    return how_are_you

In [15]:
@dsl.pipeline
def hello_pipeline(recipient: str) -> str:
    hello_task = say_hello(name=recipient)

    how_task = how_are_you(hello_text=hello_task.output)
    return how_task.output

In [16]:
compiler.Compiler().compile(hello_pipeline, 'pipeline.yaml')

In [17]:
pipeline_arguments = {
    "recipient": "Hello World"
}

In [18]:
from google.cloud.aiplatform import PipelineJob

In [20]:
job = PipelineJob(
    template_path="pipeline.yaml",
    display_name="kubeflow_pipeline",
    parameter_values=pipeline_arguments,
    location="us-central1",
    pipeline_root="./",
)

/Users/goutham/.pyenv/versions/3.9.17/envs/genai/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [22]:
job.submit()

Error when trying to get or create a GCS bucket for the pipeline output artifacts
Traceback (most recent call last):
  File "/Users/goutham/.pyenv/versions/3.9.17/envs/genai/lib/python3.9/site-packages/google/cloud/aiplatform/pipeline_jobs.py", line 432, in submit
    gcs_utils.create_gcs_bucket_for_pipeline_artifacts_if_it_does_not_exist(
  File "/Users/goutham/.pyenv/versions/3.9.17/envs/genai/lib/python3.9/site-packages/google/cloud/aiplatform/utils/gcs_utils.py", line 237, in create_gcs_bucket_for_pipeline_artifacts_if_it_does_not_exist
    pipelines_bucket = storage.Bucket.from_string(
  File "/Users/goutham/.pyenv/versions/3.9.17/envs/genai/lib/python3.9/site-packages/google/cloud/storage/bucket.py", line 753, in from_string
    raise ValueError("URI scheme must be gs")
ValueError: URI scheme must be gs
Creating PipelineJob


PermissionDenied: 403 Permission 'aiplatform.pipelineJobs.create' denied on resource '//aiplatform.googleapis.com/projects/confidently-prod/locations/us-central1' (or it may not exist). [reason: "IAM_PERMISSION_DENIED"
domain: "aiplatform.googleapis.com"
metadata {
  key: "resource"
  value: "projects/confidently-prod/locations/us-central1"
}
metadata {
  key: "permission"
  value: "aiplatform.pipelineJobs.create"
}
]

In [23]:
TRAINING_DATA_URI = "./tune_data_stack_overflow_python_qa.jsonl" 
EVAUATION_DATA_URI = "./tune_eval_data_stack_overflow_python_qa.jsonl"  

In [24]:
template_path = 'https://us-kfp.pkg.dev/ml-pipeline/\
large-language-model-pipelines/tune-large-model/v2.0.0'

In [25]:
import datetime
date = datetime.datetime.now().strftime("%H:%d:%m:%Y")

In [26]:
MODEL_NAME = f"deep-learning-ai-model-{date}"

In [28]:
TRAINING_STEPS = 200

In [48]:
EVALUATION_INTERVAL = 20
!source .env

In [49]:
from utils import authenticate
credentials, PROJECT_ID = authenticate()

In [50]:
REGION = "us-central1"

In [51]:
pipeline_arguments = {
    "model_display_name": MODEL_NAME,
    "location": REGION,
    "large_model_reference": "text-bison@001",
    "project": PROJECT_ID,
    "train_steps": TRAINING_STEPS,
    "dataset_uri": TRAINING_DATA_URI,
    "evaluation_interval": EVALUATION_INTERVAL,
    "evaluation_data_uri": EVAUATION_DATA_URI,
}

In [56]:
pipeline_root = "gs://hello-world-pipelines"

'gp-llmops-project'

In [57]:
job = PipelineJob(
    template_path=template_path,
    display_name=f"deep_learning_ai_pipeline-{date}",
    parameter_values=pipeline_arguments,
    location=REGION,
    pipeline_root=pipeline_root,
    enable_caching=True,
    credentials=credentials,
    project=PROJECT_ID
)



In [58]:
job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/438783040332/locations/us-central1/pipelineJobs/tune-large-model-20240529103436
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/438783040332/locations/us-central1/pipelineJobs/tune-large-model-20240529103436')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/tune-large-model-20240529103436?project=438783040332
